# Support Vector Regressor
Haytham Mohamed

INFS 890 - Spring 2020

In [176]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn import preprocessing, metrics, feature_selection
from datetime import datetime
from scipy import stats
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

sns.set()
sns.set(color_codes=True)
#sns.set_color_codes()

pd.options.display.max_rows = 15
pd.options.display.float_format = '{:,.2f}'.format

Set this flag to True if you're using reduction feature using FCA, otherwise set it to False

In [177]:
# 1- read data file
# 1- read processed file
home_dir = '/Users/hmohamed/github/data-research-spring2020/sock-shop'
file_dir = home_dir + '/processed-data/'



#data_file = 'order_flow_cleanup_data.csv'
data_file = 'order_flow_normalized_data.csv'
#data_file = 'order_flow_standardized_data.csv'
#data_file = 'order_flow_factored_data.csv'


test_split = 0.1


In [178]:
def read_df(file_dir, data_file, timeseries=True):
    df = pd.read_csv(file_dir + data_file)
    if timeseries:
        df = to_time_series(df)
    return df

def to_time_series(df, index_col_name='date'):
    df[index_col_name] = pd.to_datetime(df[index_col_name])
    df.set_index(index_col_name, inplace=True)
    df.sort_index(inplace=True)
    return df

def merge(df, series):
    return pd.merge_asof(df, series, left_index=True, right_index=True, tolerance=pd.Timedelta('1 second')).bfill()    


In [179]:
data = read_df(file_dir, data_file, True)
data = data.reset_index(drop=True)
data.head(5)

,front-end_cpu_use,orders_cpu_use,orders-db_cpu_use,user_cpu_use,user-db_cpu_use,shipping_cpu_use,payment_cpu_use,carts_cpu_use,carts-db_cpu_use,front-end_pods,orders_pods,user_pods,shipping_pods,payment_pods,carts_pods,nodes_cpu_use,orders_req,orders_ltcy
0,0.07,0.02,0.01,0.02,0.02,0.01,0.01,0.12,0.16,1.00,1.00,1.00,1.00,1.00,1.00,1.18,0.40,0.63
1,0.06,0.03,0.01,0.02,0.02,0.01,0.01,0.14,0.17,1.00,1.00,1.00,1.00,1.00,1.00,1.18,0.44,0.62
2,0.06,0.03,0.01,0.02,0.02,0.01,0.01,0.15,0.14,1.00,1.00,1.00,1.00,1.00,1.00,1.15,0.36,0.62
3,0.09,0.03,0.01,0.02,0.02,0.01,0.01,0.06,0.19,1.00,1.00,1.00,1.00,1.00,1.00,1.54,0.40,0.56
4,0.11,0.04,0.01,0.03,0.03,0.01,0.01,0.11,0.32,1.00,1.00,1.00,1.00,1.00,1.00,2.35,0.78,0.53


In [180]:
data.shape

(1499, 18)

In [181]:
targets = data['orders_ltcy']
inputs = data.drop(['orders_ltcy'], axis=1)

n_features = inputs.values.shape[1]

#inputs_scaled = scaler.fit_transform(inputs)
#x_train, x_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=365)

x_train, x_test, y_train, y_test = \
    train_test_split(inputs, targets, test_size=test_split, shuffle=False, random_state=365)

# for better convergence and result scale target to values between 0 - 1
y_train_max = y_train.max()
y_test_max = y_test.max()

y_train = y_train / y_train_max
y_test = y_test / y_test_max

print("there are {} features".format(n_features))


there are 17 features


# Feature Selection

In [182]:
x_train_select=x_train
y_train_select=y_train
x_train_select.shape


(1349, 17)

In [183]:
# -- trying features selection 1
if 1 == 0:
    selector = feature_selection.SelectKBest(feature_selection.f_regression).fit(x_train_select, y_train_train)
    selectedFeatures = selector.transform(x_train_train)
    print('{} features selected'.format(selectedFeatures[1].shape[0]))
    

In [184]:
# -- trying features selection 2
if 1 == 0:

    thresholds = ['3*mean', '2*mean', 'mean', '0.25*mean', '0.05*mean']
    #max_features=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
    
    selection = 4
    
    lasso = LassoCV(fit_intercept=False, tol=0.00000000000001,n_jobs=-1,random_state=365
                    ,selection='random', max_iter=1000000000, positive=False, cv=10)

    #sfm = SelectFromModel(lasso, threshold=-np.inf, max_features=max_features[3]) #(reg, threshold=0.01)
    sfm = SelectFromModel(lasso, threshold=thresholds[selection]) #(lasso, threshold=0.01)

    selectedFeatures = sfm.fit(x_train_select, y_train_select).transform(x_train_select)
    print('{} features selected'.format(selectedFeatures[1].shape[0]))
    

In [185]:
# -- trying features selection 3
if 1 == 0:
    thresholds = ['1*mean', '0.8*mean', '0.6*mean', '0.3*mean', '0.1*mean','0.05*mean', '0.02*mean', '0.01*mean']
    clf = RandomForestRegressor(n_jobs=-1, max_depth=10, n_estimators=15)

    sfm = SelectFromModel(clf,threshold=thresholds[0])

    selectedFeatures = sfm.fit(x_train_select, y_train_select).transform(x_train_select)
    print('{} features selected'.format(selectedFeatures[1].shape[0]))

In [186]:
#Features selected
if 1 == 0:
    feature_list = x_train.columns[sfm.get_support()]
    features = ''
    features = ', '.join(feature_list)
    #features
    
    for feature in feature_list:
        print(feature)

    reduced_x_train = x_train[feature_list]
    reduced_x_train.head(2)

# Model

In [188]:
# start:  search grid

# SVR
svr = LinearSVR(random_state=365, verbose=0)   #(C=0.1, random_state=365, verbose=0)

# standardize the features
scaler = StandardScaler()

# features selection
lasso = LassoCV(fit_intercept=False, tol=0.00000000000001,n_jobs=-1,random_state=365
                    ,selection='random', max_iter=1000000000, positive=False, cv=10)
    
sfm = SelectFromModel(lasso)

pipe = Pipeline(steps=[
    ('scaler', scaler),
    ('selector', sfm), 
    ('clf', svr)
])

kfold = KFold(n_splits=2, shuffle=True)

param_grid = {
    'selector__threshold': ['3*mean', '2*mean', 'mean', '0.25*mean', '0.05*mean']
    #'selector__max_features': [3, 7, 10, 13, 17]
    ,'clf__C': [.01, .05, .1]
}

search = GridSearchCV(pipe
                      , param_grid,scoring='neg_mean_squared_error'
                      , return_train_score= True
                      , cv=kfold
                      , verbose=1
                      , n_jobs=-1)

search.fit(x_train, y_train)


Fitting 2 folds for each of 15 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
estimator = search.best_estimator_
params = search.best_params_
score = search.best_score_

print("Best CV score = %0.3f:" % score)
print("Best CV params = {}".format(params))

print("")
all_training_scores = search.cv_results_
#print("\ncross_validation:\n", all_training_scores , "\n")
print(all_training_scores['mean_train_score'])

print("")
print('Model training Score = {}'.format(search.score(x_train,y_train)))


# Features Selected

In [ ]:
sfm = pipe['selector']

selectedFeatures = sfm.fit(x_train_select, y_train_select).transform(x_train_select)
print('{} features selected'.format(selectedFeatures[1].shape[0]))

feature_list = x_train_select.columns[sfm.get_support()]
features = ''
features = ', '.join(feature_list)
#features

reduced_x_train = x_train_select[feature_list]
reduced_x_train.head(2)

In [ ]:
pdf = pd.DataFrame(all_training_scores['mean_train_score'], columns=['training mse'])
pdf.plot()

# Training R2

In [ ]:
y_hat = search.predict(x_train)

r2_train = metrics.r2_score(y_train, y_hat)
print('R2 (training) = {}'.format(r2_train))


In [ ]:
# validation mean_absolute_percentage_error
train_error =  np.abs(y_train - y_hat)
mean_error = np.mean(train_error)
min_error = np.min(train_error)
max_error = np.max(train_error)
std_error = np.std(train_error)


print("Model Result (mean - std of error - min - max): %.2f (%.2f) %.2f %.2f MSE" \
      % (mean_error, std_error, min_error, max_error))

In [ ]:
plt.scatter(y_train, y_hat)
plt.xlabel('Latency', size=18)
plt.ylabel("predicted Latency", size=18)
#plt.xlim(-2,3)
#plt.ylim(-3,1)
plt.show()

In [ ]:
sns.distplot(y_train - y_hat)
plt.title('Residual PDF', size=18)

# Testing

In [ ]:
y_hat_test = search.predict(x_test)

In [ ]:
plt.scatter(y_test, y_hat_test)
plt.xlabel('Latency', size=18)
plt.ylabel('Predicted Latency', size=18)
plt.show

In [ ]:
ax = sns.regplot(x=y_test, y=y_hat_test, color='g')

In [ ]:
perf = pd.DataFrame(np.exp(y_hat_test), columns=['prediction'])
y_test = y_test.reset_index(drop=True)
perf['target'] = np.exp(y_test)
perf.head()

In [ ]:
# testing mean_absolute_percentage_error
perf['residual'] = perf['prediction'] - perf['target']
perf['difference%'] = np.absolute(perf['residual'] * 100 / perf['target'])
perf

In [ ]:
perf.describe()

In [ ]:
pd.options.display.max_rows = 999
pd.set_option('display.float_format', lambda x: '%.2f' % x)
perf.sort_values(by = ['difference%'])

# *** Prediction Accuracy

In [ ]:
r2_test = metrics.r2_score(y_test, y_hat_test)
print('R2 (testing) = {}'.format(r2_test))

#also: R-Squared could obtain with
#pipe.score(x_test, y_test)

In [ ]:
print(np.sqrt(mean_squared_error(y_test,y_hat_test)))